# <b>Data Visualization </b> *✲ﾟ*｡✧٩(･ิᴗ･ิ๑)۶*✲ﾟ*｡✧

Now that we have the processed and imputed data, we will project it into a lower space with PCA (and potentially ICA and t-SNE) and visualize it such that we see potential outliers but also certain patterns that might seem odd. 

In [ ]:
import helper_functions as hf
import numpy as np
import plotly.graph_objects as go
import numpy as np
import imageio
import os
import pandas as pd
import importlib
importlib.reload(hf)

imputed_dataframe = pd.read_hdf("imputed_dataframe_0602.h5",key="data")


# Dimensionality Reduction

### Performing PCA on imputed data

In [ ]:
pca = hf.PCA(n_components=3)
pca_imputed_data = hf.pd.DataFrame(pca.fit_transform(imputed_dataframe.loc[:,~imputed_dataframe.columns.isin(['state', 'dataset'])]))
pca_imputed_data.to_pickle('pca_imputed_data.pkl')

### OLD AND NOT NEEDED: Getting the behavioural states of each observation
We compute the behavioural state of the worm given an observation via the neuron "AVA" for reversal, the neurons "SMDV" and "RIM" for ventral turn and "SMDD" for dorsal turn. 
If the state of AVA for an observation is "high" or "rise" (based on some activity threshold) then that observation gets the state "reversal" assigned. If the state of AVA is "fall", then we look at the turn neurons. Whichever one has a "rise" state for a longer period of time gets assigned to the observation, so in case of "SMDV" -> ventral, "SMDD" -> dorsal. If both are "fall", then the observation gets the state "forward".

### Plotting the PCs

In [ ]:
annotations = imputed_dataframe['state']
pca_imputed_data['state'] = annotations

hf.plot_PCs(pca_imputed_data, 'PCA of Time Embedded Data', pca.explained_variance_ratio_*100)

In above visualization of the principal components of all >88k observations we can observe some patterns. The are some data points assigned to the ventral state that are very far away from the rest of the data. In 'data_diagnostics.ipynb' we will try to locate outliers and see if these ventral points are among them. If that is the case we also want to know if they belong to the same dataset or if they are spread across all datasets. Another observation is that there is an L-shape in the forward state observations but maybe also in the reversal ones. The question is what the cause of this is. More on this in the data diagnostics notebook.

## Appendix I: Other Dimensionality Reduction Techniques

### I.1 Locally Linear Embedding (LLE)

We test LLE which is an unsupervised learning algorithm that computes low-dimensional, neighborhood-preserving embeddings of high-dimensional inputs. It is a manifold learning technique that does not rely on the assumption that the manifold is globally linear.

In [ ]:
# Apply Locally Linear Embedding (LLE) for manifold untangling
from sklearn.manifold import LocallyLinearEmbedding


lle = LocallyLinearEmbedding(n_neighbors=12, n_components=3, method='standard')
X_lle = lle.fit_transform(imputed_dataframe)
X_df = hf.pd.DataFrame(X_lle)

X_df['state'] = annotations.values
ll_df,names2 = hf.utils_plot_traces.modify_dataframe_to_allow_gaps_for_plotly(X_df, [0,1,2] ,'state')

In [ ]:
phase_plot_list = []
for i, state_code in enumerate(state_codes):
    phase_plot_list.append(
                go.Scatter3d(x=ll_df[names[0][i]], y=ll_df[names[1][i]], z=ll_df[names[2][i]],
                                name=state_code))

fig = go.Figure()
fig.add_traces(phase_plot_list)
fig.write_html("LLE_exploration.html")
fig.show()

### I.2 ICA
**ICA (Independent Component Analysis)** tries to find a set of independent basis vectors that represent the data. PCA tries to find uncorrelated vectors that capture the variance of the data the most and these vectors have to be orthogonal to each other, but ICA does not care about capturing the most information it just wants to separate different signals/features into independent additive subcomponents. ICA usually finds usage with physical signals but it can be extended to non-physical signals.  

In [ ]:
ica = hf.FastICA(n_components=3,
        random_state=0,
        whiten='unit-variance').fit_transform(imputed_dataframe)
ica_annotated_data_df = hf.pd.DataFrame(ica)
ica_annotated_data_df['state'] = annotations.values
ica_df,ica_names = hf.utils_plot_traces.modify_dataframe_to_allow_gaps_for_plotly(ica_annotated_data_df, [0,1,2] ,'state')

phase_plot_list = []
for i, state_code in enumerate(state_codes):
    phase_plot_list.append(
                go.Scatter3d(x=ica_df[names[0][i]], y=ica_df[names[1][i]], z=ica_df[names[2][i]], mode='lines',
                                name=state_code))

fig = go.Figure()
fig.add_traces(phase_plot_list)
fig.write_html("ICA_exploration.html")
fig.show()

## Appendix I: PCA of Transpose 
In order to find out which neurons are explained most by the data, we can perform PCA on the transpose such that the neurons are the "observations" and the trials are the "features". Take neurons AVAL and RIML for example: If the weights of PC1 are higher for "observation" AVAL than for RIML, then the trials that contribute to PC1 have a stronger influence on the variation in AVAL than in RIML. 

In [ ]:
imputed_transposed_dataframe = imputed_dataframe.T
n_components = 3

pca_all_splits = hf.get_LLO_PCAs(imputed_transposed_dataframe, n_components=n_components)

fig = hf.make_subplots(rows=3, cols=1, shared_xaxes=True, y_title= "PCA weights", vertical_spacing=0.05)

for i in range(n_components):
    variable_name = f"pca{i+1}_all_splits"
    concatenated_pca = hf.pd.concat(pca_all_splits[variable_name], axis=0)
    concatenated_pca.sort_values(by=['neuron'], inplace=True)
    fig.add_trace(hf.go.Box(
        x=concatenated_pca['neuron'],
        y=concatenated_pca['Mode {}'.format(i+1)],
        boxpoints=False,
        name='Mode {}'.format(i+1)
    ), row=i+1, col=1)


# Update layout
fig.update_layout(
    title_text="First 3 PC weights for all neurons",
    height=600
)

# Show the figure
fig.show()
fig.write_html("PCA_neuron_weights.html")